<a href="https://colab.research.google.com/github/MihneaGherghel/ColabCopy/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import re
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
class OptainData:
  def __init__(self,file_path):
    self.data = pd.read_csv(file_path)
    self.data['label'] = pd.to_numeric(self.data['label'], errors='coerce')
    self.data = self.data.dropna()
    self.data['label'] = self.data['label'].astype(int)
    self.data.reset_index(inplace=True)

  def split_labels_samples(self):
    samples=self.data.drop('label',axis=1)
    labels=self.data['label']
    return samples,labels,self.data

In [52]:
data=OptainData('./gdrive/MyDrive/train.csv/FakeNews.csv')
samples,labels,samples_labels=data.split_labels_samples()

In [53]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [55]:
messages=samples_labels.copy()
messages.reset_index(inplace=True)
corpus=[]
ps=PorterStemmer()
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [66]:
#BagOfWords
cv=CountVectorizer(max_features=5000, ngram_range=(1,3))
x=cv.fit_transform(corpus).toarray()

In [61]:
#Tfidf

tfidf=TfidfVectorizer(max_features=5000, ngram_range=(1,3))
x=tfidf.fit_transform(corpus).toarray()

In [68]:
y=samples_labels['label']

In [69]:
X_train,X_test, y_train,y_test=train_test_split(x,y, test_size=0.33, random_state=0)

In [70]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import itertools

In [71]:
classifier=MultinomialNB()
classifier.fit(X_train,y_train)
pred=classifier.predict(X_test)
score=metrics.accuracy_score(y_test,pred)
print(score)

0.9019055509527755
